In [ ]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

class Simulation:
    def generate_interarrival(self):  
        ### Generate exponentially distributed arrival Time
        return  (-1*self.arrival_time*math.log(random.random()))  

    def generate_service(self):        
        ### Generate exponentially distributed service Time
        return  (-1*self.service_time*math.log(random.random())) 
    
    def generate_handover(self):
        ### Generate exponentially distributed handover Time
        return  (-1*self.handover_time*math.log(random.random())) 
    
    def initialize(self):        
        ### Initialize the simulation objects 
        
        self.server_status      = [0 for i in range(self.no_of_servers)]
        self.area_server_status = [0 for i in range(self.no_of_servers)]
        self.server_utilization = [0 for i in range(self.no_of_servers)]
        self.time_next_event    = [0 for i in range(self.no_of_servers + 2)]        
            
        self.clock_time = 0.0                      
        self.time_last_event = 0.0
        self.num_events = self.no_of_servers + 2
        
        self.lost_calls = 0
        self.lost_handovers = 0
        self.total_arrivals = 0
        self.total_departed = 0                
        self.total_handovers = 0        
        self.total_server_utilization = 0.0
        self.arrival_blocking_probability = 0.0    
        self.handover_blocking_probability = 0.0        
        self.calls_blocked_due_to_threshold_limit = 0        
        
        for i in range(len(self.time_next_event)):
            if i == 0:
                self.time_next_event[i] = self.generate_interarrival()
            elif i == 1:
                self.time_next_event[i] = self.generate_handover()
            else:
                self.time_next_event[i] = float('inf')

    def timing(self):
        ### Advances the time for simulation and calculate the next event type
        ## Determine the event type of the next event to occur
        
        self.min_time_next_event = float('inf')
        
        for i in range(self.num_events):
            if (self.time_next_event[i] <= self.min_time_next_event):
                self.min_time_next_event = self.time_next_event[i]
                self.next_event_type = i

        self.time_last_event = self.clock_time
        
        ## advance the simulation clock
        self.clock_time = self.time_next_event[self.next_event_type]

    def arrive(self):
        ### Schedule next call arrival, If Idle server is present 
        ### and idle servers are more than 2 then only service New Arrivals
        ### else not, as threshold limit is set to 2
        
        self.time_next_event[0] = self.clock_time + self.generate_interarrival()
        self.total_arrivals += 1
        
        ## Find out the idle server
        i = 0
        server_idle = -1
        
        idle_server_count =  self.no_of_servers - sum(self.server_status)
        
        while (server_idle == -1 and i < self.num_events - 2 ):
            if (self.server_status[i] == 0):
                server_idle = i  ## Found idle server
                break
            i += 1

        if server_idle >= 0 and idle_server_count > 2:   ## Threshold is 2 here 
            self.server_status[server_idle] = 1
            self.time_next_event[server_idle + 2] = self.clock_time +  self.generate_service() 

        else:  ## servers are BUSY, new call is lost 
            self.lost_calls += 1
            if idle_server_count <= 2:
                self.calls_blocked_due_to_threshold_limit += 1
            
    def handover(self):
        ## Schedule next handover
        self.time_next_event[1] = self.clock_time + self.generate_handover()
        self.total_handovers += 1
        
        ## Find out the idle server
        i = 0
        server_idle = -1
        
        while (server_idle == -1 and i < self.num_events - 2):
            if (self.server_status[i] == 0):
                server_idle = i  ## Found idle server
                break
            i += 1

        if server_idle >= 0:  ## Found idle server
            self.server_status[server_idle] = 1
            self.time_next_event[server_idle + 2] = self.clock_time +  self.generate_service()

        else:  ## server is BUSY, handover is lost
            self.lost_handovers += 1
            #self.lost_calls += 1
    
    def depart(self, j):
        ### Departure of the call
        
        self.server_status[j - 2] = 0
        self.time_next_event[j] = float('inf')
        self.total_departed += 1

    def update_time_avg_stats(self):
        ## Update area accumulators for time-average statistics
        
        self.time_past = self.clock_time - self.time_last_event

        for i in range(self.no_of_servers):
            self.area_server_status[i] += self.time_past * self.server_status[i]

    def report(self):
        ## Compute the desired measures of performance
        ### i.e Simulation's server utilization and different blocking probabilities
        
        self.arrival_blocking_probability = self.lost_calls / self.total_arrivals
        
        self.handover_blocking_probability = self.lost_handovers / self.total_handovers

        for i in range(self.no_of_servers):
            self.server_utilization[i] = self.area_server_status[i] / self.clock_time
            self.total_server_utilization += self.area_server_status[i]

        self.total_server_utilization = self.total_server_utilization /self.clock_time/ self.no_of_servers
        
    def main(self, trial):
        ### Main method to start the simulation
        
        self.no_of_servers           = 16
        self.num_calls_required      = 100000
        self.service_time            = 100
        self.handover_time           = 30
        
        if trial == 0:
            self.arrival_time        = 10   #ideal 12.8
        else:
            self.arrival_time        = 25
        
        self.initialize();  ## Initialize the simulation

        while (self.total_arrivals + self.total_handovers < self.num_calls_required):
            self.timing()  ## Determine the next event            

            if (self.next_event_type == 0):
                self.arrive()  ## next event is arrival
            elif(self.next_event_type == 1):
                self.handover()  ## next event is handover
            else:
                self.depart(self.next_event_type) ## next event is departure
            
            self.update_time_avg_stats()
            
        self.report();

for x in range(2):         # Two trials with different arrival rate, just to plot and compare the performance
    np.random.seed(0)
    s = Simulation()
    s.main(x)
    lamda1 = 1/s.arrival_time
    lamda2 = 1/s.handover_time
    mu     = 1/s.service_time
    c      = s.no_of_servers
    n      = 2 
    Po     = 0.0    
    Po1    = 0.0    
    Po2    = 0.0  
    Pk     = 0.0
    Pk1    = 0.0    
    Pk2    = 0.0 
    
    for k in range((c + 1) -n):    ## Po1 for 0 <= k <= C-N
        Po1 = (1/math.factorial(k)) * (((lamda1 + lamda2) / mu)**k) 
    
    for k in range(15,17):        ## Po2 for C-N+1 <= k <= C 
        Po2  = (1/math.factorial(c-n)) * (((lamda1 + lamda2)/mu)**(c-n)) * (1/math.factorial(k-(c-n))) * ((lamda2/mu)**(k -(c-n)))
        #Po2  = (1/math.factorial(k)) * (((lamda1 + lamda2)/mu)**(c-n)) * ((lamda2/mu)**(k -(c-n)))
    
    Po = 1/(Po1 + Po2)
    
    for k in range((c + 1) -n):  ## Pk1 for 0 <= k <= C-N
        Pk1 = (1/math.factorial(k)) * (((lamda1 + lamda2)/mu)**k) * (Po)
    
    for k in range(15,17):       ## Pk2 for C-N+1 <= k <= C
        Pk2 = (1/math.factorial(k)) * (((lamda1 + lamda2)/mu)**(c-n)) * ((lamda2/mu)**(k-(c-n))) * (Po)
    
    numerator_for_Pc   = 0.0
    denominator_for_Pc = 0.0
    math_blocking_prob = 0.0 
    
    numerator_for_Pc   = (((lamda1 + lamda2)/mu)**c)/math.factorial(c)
    
    for i in range(c + 1):
        denominator_for_Pc +=  ((((lamda1 + lamda2)/mu)**i)/math.factorial(i))
    
    math_blocking_prob = numerator_for_Pc/denominator_for_Pc     
    
    print("*******************Arrival rate = " +str(1/s.arrival_time) + "**********************")
    print("Handover Rate                         = " +str(1/s.handover_time))
    print("Service  Rate                         = " +str(1/s.service_time))
    print("Number of calls                       = " +str(s.num_calls_required))    
    print("Servers Utilization                   = " + str(s.total_server_utilization))
    print("New Arrivals                          = " + str(s.total_arrivals))
    print("New Arrivals successful               = " + str(s.total_arrivals - s.lost_calls))
    print("New Handovers                         = " + str(s.total_handovers))    
    print("Departures                            = " + str(s.total_departed + sum(s.server_status)))
    print("Lost Arrivals                         = " + str(s.lost_calls))
    print("Lost Handovers                        = " + str(s.lost_handovers))
    print("Total Lost Calls                      = " + str(s.calls_blocked_due_to_threshold_limit + s.lost_handovers))
    print("Simulation New Arrival Blocking Probability(CBP) = " + str(s.arrival_blocking_probability))
    print("Math New Arrivals Blocking Probability           = " + str(Pk1))
    print("Simulation Handover Failure Probability(HFP)     = " + str(s.handover_blocking_probability))
    print("Math Handover Failure Probability                = " + str(Pk2))
    print("Aggregated Blocking Probability(ABP)             = " + str(s.arrival_blocking_probability + 10 * s.handover_blocking_probability))
    print("\n")

    num_of_servers  = [i+1 for i in range(s.no_of_servers)]
    plt.scatter(num_of_servers, s.server_utilization, label = 'Arrival Time - '+str(s.arrival_time))
    
plt.title('Server Utilization vs No. of Servers')
plt.xlabel('No. of Servers')
plt.ylabel('Server Utilization')
plt.legend()
plt.show() 